<a href="https://colab.research.google.com/github/DiaconuDenis/Movie-Recommendation-System-Using-RBM/blob/main/Boltzmann_Machine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing the libraries

In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

# Importing the dataset

In [ ]:
movies = pd.read_csv('ml-1m/movies.dat', sep='::', header=None,engine='python', encoding= 'latin-1')
users = pd.read_csv('ml-1m/users.dat', sep='::', header=None,engine='python', encoding= 'latin-1')
ratings = pd.read_csv('ml-1m/ratings.dat', sep='::', header=None,engine='python', encoding= 'latin-1')

# Preparing the training test and the test set

In [ ]:
training_set = pd.read_csv('ml-100k/u1.base', delimiter= '\t')
training_set = np.array(training_set, dtype= 'int')
test_set = pd.read_csv('ml-100k/u1.test', delimiter= '\t')
test_set = np.array(test_set, dtype= 'int')

# Getting the number of users and movies

In [ ]:
nb_users = int(max(max(training_set[:,0]), max(test_set[:,0])))
nb_movies = int(max(max(training_set[:,1]), max(test_set[:,1])))

# Converting the data into an array with users in lines and movies in columns

In [ ]:
def convert(data):
  new_data = []
  for id_users in range(1, nb_users+1):
    id_movies = data[:,1][data[:,0] == id_users]
    id_ratings = data[:,2][data[:,0] == id_users]
    ratings = np.zeros(nb_movies)
    ratings[id_movies - 1] = id_ratings
    new_data.append(list(ratings))
  return new_data
training_set = convert(training_set)
test_set = convert(test_set)

# Converting the data into Torch tensors

In [ ]:
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

# Converting the ratings into binary ratings 1(Liked) or 0(Not Liked)

In [ ]:
training_set[training_set == 0] = -1
training_set[training_set == 1] = 0
training_set[training_set == 2] = 0
training_set[training_set >= 3] = 1

test_set[test_set == 0] = -1
test_set[test_set == 1] = 0
test_set[test_set == 2] = 0
test_set[test_set >= 3] = 1

# Creating the architecture of the Neural Network

In [ ]:
class RBM():
  def __init__(self, nv, nh):
    self.W = torch.randn(nh, nv)
    self.a = torch.randn(1, nh)
    self.b = torch.randn(1, nv)

  def sample_h(self, x):
    wx = torch.mm(x, self.W.t())
    activation = wx + self.a.expand_as(wx) # Changed expend_as to expand_as
    p_h_given_v = torch.sigmoid(activation)
    return p_h_given_v, torch.bernoulli(p_h_given_v)

  def sample_v(self, y):
    wy = torch.mm(y, self.W)
    activation = wy + self.b.expand_as(wy) # Changed expend_as to expand_as
    p_v_given_h = torch.sigmoid(activation)
    return p_v_given_h, torch.bernoulli(p_v_given_h)

  def train(self, v0, vk, ph0, phk):
    self.W += torch.mm(ph0.t(), v0) - torch.mm(phk.t(), vk)
    self.b += torch.sum((v0 - vk), 0)
    self.a += torch.sum((ph0 - phk), 0)


nv = len(training_set[0])
nh = 100
batch_size = 100

# Creating a RBM object

In [ ]:
rbm = RBM(nv, nh)

# Training the RBM

In [ ]:
nb_epoch = 10
for epoch in range(1, nb_epoch + 1):
  train_loss = 0
  s = 0.
  for id_user in range(0, nb_users - batch_size, batch_size):
    vk = training_set[id_user:id_user + batch_size]
    v0 = training_set[id_user:id_user + batch_size]
    ph0,_ = rbm.sample_h(v0)
    for k in range(10):
      _,hk = rbm.sample_h(vk)
      _,vk = rbm.sample_v(hk)
      vk[v0 < 0] = v0[v0 < 0]
    phk,_ = rbm.sample_h(vk)
    rbm.train(v0, vk, ph0, phk)
    train_loss += torch.mean(torch.abs(v0[v0 >= 0] - vk[vk >= 0]))
    s += 1.
    print('epoch: '+str(epoch)+ ' loss: '+str(train_loss/s))

epoch: 1 loss: tensor(0.5115)
epoch: 1 loss: tensor(0.5728)
epoch: 1 loss: tensor(0.5083)
epoch: 1 loss: tensor(0.4602)
epoch: 1 loss: tensor(0.4267)
epoch: 1 loss: tensor(0.4231)
epoch: 1 loss: tensor(0.3993)
epoch: 1 loss: tensor(0.3906)
epoch: 1 loss: tensor(0.3750)
epoch: 2 loss: tensor(0.2713)
epoch: 2 loss: tensor(0.2616)
epoch: 2 loss: tensor(0.2523)
epoch: 2 loss: tensor(0.2465)
epoch: 2 loss: tensor(0.2475)
epoch: 2 loss: tensor(0.2501)
epoch: 2 loss: tensor(0.2523)
epoch: 2 loss: tensor(0.2550)
epoch: 2 loss: tensor(0.2566)
epoch: 3 loss: tensor(0.2422)
epoch: 3 loss: tensor(0.2333)
epoch: 3 loss: tensor(0.2279)
epoch: 3 loss: tensor(0.2258)
epoch: 3 loss: tensor(0.2309)
epoch: 3 loss: tensor(0.2365)
epoch: 3 loss: tensor(0.2412)
epoch: 3 loss: tensor(0.2458)
epoch: 3 loss: tensor(0.2459)
epoch: 4 loss: tensor(0.2474)
epoch: 4 loss: tensor(0.2467)
epoch: 4 loss: tensor(0.2398)
epoch: 4 loss: tensor(0.2312)
epoch: 4 loss: tensor(0.2371)
epoch: 4 loss: tensor(0.2455)
epoch: 4 l

# Testing the RBM

In [ ]:

test_loss = 0
s = 0.
for id_user in range(nb_users):
  v = training_set[id_user:id_user+1]
  vt = test_set[id_user:id_user+1]
  if len(vt[vt>=0]) > 0:
    _,h = rbm.sample_h(v)
    _,v = rbm.sample_v(h)
    test_loss += torch.mean(torch.abs(vt[vt >= 0] - v[vt >= 0]))
    s += 1.
  print('test loss: '+str(test_loss/s))

test loss: tensor(0.1838)
test loss: tensor(0.2055)
test loss: tensor(0.2652)
test loss: tensor(0.2489)
test loss: tensor(0.2658)
test loss: tensor(0.2545)
test loss: tensor(0.2392)
test loss: tensor(0.2265)
test loss: tensor(0.2014)
test loss: tensor(0.1901)
test loss: tensor(0.1956)
test loss: tensor(0.1889)
test loss: tensor(0.2007)
test loss: tensor(0.1976)
test loss: tensor(0.2163)
test loss: tensor(0.2080)
test loss: tensor(0.2154)
test loss: tensor(0.2110)
test loss: tensor(0.2051)
test loss: tensor(0.2153)
test loss: tensor(0.2238)
test loss: tensor(0.2230)
test loss: tensor(0.2202)
test loss: tensor(0.2141)
test loss: tensor(0.2077)
test loss: tensor(0.2082)
test loss: tensor(0.2079)
test loss: tensor(0.2058)
test loss: tensor(0.2109)
test loss: tensor(0.2168)
test loss: tensor(0.2197)
test loss: tensor(0.2195)
test loss: tensor(0.2189)
test loss: tensor(0.2183)
test loss: tensor(0.2228)
test loss: tensor(0.2217)
test loss: tensor(0.2197)
test loss: tensor(0.2230)
test loss: t